<a href="https://colab.research.google.com/github/Sahel-Eskandar/Project-Notebooks/blob/main/5_Transfer_leaning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import keras
import numpy as np
from PIL import Image
import tensorflow as tf
from random import randint
import matplotlib.pyplot as plt
from keras import backend as k
from keras.utils import np_utils
from keras.optimizers import adam
from keras.models import Sequential
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.image import img_to_array
from keras.metrics import categorical_crossentropy
#from sklearn.cross_validation import train_test_split
from keras.layers import Dense,Dropout,Activation,Flatten,Conv2D
from keras.layers.convolutional import Convolution2D,MaxPooling2D

#from keras import backend as K
#K.set_image_dim_ordering('th')


Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

SyntaxError: ignored

In [ ]:
import os
import pprint
if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; ')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

In [ ]:
#process for reproducing same result
import os
import random as rm

# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/fchollet/keras/issues/2280#issuecomment-306959926
os.environ['PYTHONHASHSEED']='0'

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(1)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
rm.seed(1)

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed
tf.set_random_seed(1)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of
# non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res
from keras import backend as k
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

sess=tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph())
k.set_session(sess)

In [ ]:
pp='/content/drive/My Drive/app1/'
train_path=pp+'cat-and-dog/training_set/'
valid_path=pp+'cat-and-dog/valid_set/'
test_path=pp+'cat-and-dog/test_set/'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
img_size=224
datagen = ImageDataGenerator(rescale=1./255)
train_batches=datagen.flow_from_directory(
               train_path,
               target_size=(img_size,img_size),
               classes=['dogs','cats'],
               batch_size=10,
               color_mode="grayscale")
valid_batches=datagen.flow_from_directory(
               valid_path,
               target_size=(img_size,img_size),
               classes=['dogs','cats'],
               batch_size=10,
               color_mode="grayscale")
test_batches=datagen.flow_from_directory(
               test_path,
               target_size=(img_size,img_size),
               classes=['dogs','cats'],
               batch_size=4,
               color_mode="grayscale")

# Build VGG16 model-Sequential

VGG16 works with activation="softmax" which does not works here.

In [ ]:
#vgg16_model=keras.applications.vgg16.VGG16()
vgg16_model = keras.applications.VGG16(weights='imagenet', input_shape = (3,224,224))

In [ ]:
vgg16_model.summary()


vgg16 is a model not a sequential model. It needs to be transformed to a sequential model.
Sequential model create models layer-by-layer for most problems. It is limited in that it does not allow you to create models that share layers or have multiple inputs or outputs.

In [ ]:
type(vgg16_model)


In [ ]:
model=Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)
    model.trainable=False  

Remove the last layer of 1000 categories and add the current selection

In [ ]:
model.add(Dense(units=2))
model.add(Activation(tf.nn.softmax))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit_generator(
                generator=train_batches,
                validation_data=valid_batches,
                steps_per_epoch=4,#train_batches.samples(8005)/train_batches.batch_size(50),
                validation_steps=4,#valid_batches.samples/valid_batches.batch_size,
                verbose=2,
                #shuffle=False,
                epochs=3)

# Prediction

In [ ]:
from keras.preprocessing.image import load_img
# load an image from file
image_path='dog.6.jpg'
image = load_img(image_path, target_size=(224, 224))

In [ ]:
np.shape(image)

In [ ]:
from keras.preprocessing.image import img_to_array
# convert the image pixels to a numpy array
image = img_to_array(image)

In [ ]:
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image = preprocess_input(image)

In [ ]:
#from keras.applications.vgg16 import decode_predictions
# predict the probability across all output classes
yhat = model.predict(image)
yhat

In [ ]:
#from keras.applications.vgg16 import decode_predictions
# convert the probabilities to class labels
#label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
#label = label[0][0]
# print the classification
#print('%s (%.2f%%)' % (label[1], label[2]*100))

# Build VGG16 model-Freez layers

In [ ]:
#keras.applications.vgg16.VGG16()
#include_top=False ---> exclude the last layer of the VGG model.
vgg_model = keras.applications.VGG16(weights='imagenet', include_top=False, input_shape = (3,224,224))

# Creating dictionary that maps layer names to the layers
layer_dict = dict([(layer.name, layer) for layer in vgg_model.layers])

In [ ]:
# Getting output tensor of the last VGG layer that we want to include
x = layer_dict['block5_pool'].output

# Stacking a new simple convolutional network on top of it    
#x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
#x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2, activation=tf.nn.softmax)(x)

# Creating new model. Please note that this is NOT a Sequential() model.
from keras.models import Model
custom_model = Model(input=vgg_model.input, output=x)

# Make sure that the pre-trained bottom layers are not trainable
for layer in custom_model.layers[:-3]:
    layer.trainable = False

# Do not forget to compile it
custom_model.compile(loss='categorical_crossentropy',
                     optimizer='rmsprop',
                     metrics=['accuracy'])

custom_model.fit_generator(
                generator=train_batches,
                validation_data=valid_batches,
                steps_per_epoch=4,#train_batches.samples(8005)/train_batches.batch_size(50),
                validation_steps=4,#valid_batches.samples/valid_batches.batch_size,
                verbose=2,
                #shuffle=False,
                epochs=3)

In [ ]:
yhat = custom_model.predict(image)
yhat

# Build VGG16 model-Initial weight

In [ ]:
#keras.applications.vgg16.VGG16()
vgg_model = keras.applications.VGG16(weights='imagenet', include_top=False, input_shape = (3,224,224))

for layer in vgg_model.layers[:5]:
    layer.trainable = True

for layer in vgg_model.layers:
    print(layer, layer.trainable)
    

In [ ]:
#Adding custom Layers 
x = vgg_model.output
x = Flatten()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(2, activation="softmax")(x)

model_final = Model(input = vgg_model.input, output = predictions)

In [ ]:
# creating the final model 

# Do not forget to compile it
model_final.compile(loss='categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])


model_final.fit_generator(
                generator=train_batches,
                validation_data=valid_batches,
                steps_per_epoch=4,#train_batches.samples/train_batches.batch_size,,
                validation_steps=4,#valid_batches.samples/valid_batches.batch_size,,
                verbose=2,
                #shuffle=False, 
                epochs=3)

# Data Augmentation

Data Augmentation occures when creating new data with modification.
Filpp, Rotate, Zoom,Color change,Cropping

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import misc,ndimage
import keras
from keras import backend as k
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
gen=ImageDataGenerator(rotation_range=10,width_shift_range=0.1,
                       height_shift_range=0.1,shear_range=0.15,
                       zoom_range=0.1,channel_shift_range=10,
                       horizontal_flip=True)

In [ ]:
import cv2
image_path='/Users/atousa/Desktop/McMaster/BDA-102/Neural_network/data/cat-and-dog/dog.6.jpg'

In [ ]:
image=cv2.imread(image_path)

In [ ]:
np.shape(image)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image[:,:,0], cmap='gray')
plt.show()

generate batches of image

In [ ]:
np.shape(image)

In [ ]:
image = np.moveaxis(image, -1, 0)
image.shape

In [ ]:
image_batch = image.reshape((1,) + image.shape)

In [ ]:
np.shape(image_batch)

Get 10 sample of the batches

In [ ]:
aug_iter=gen.flow(image_batch, batch_size=2)

In [ ]:
aug_images=[next(aug_iter)[0].astype(np.uint8) for i in range(10)]
np.shape(aug_images)

In [ ]:
for i in range(0,10):
    img = aug_images[i]
    plt.imshow(img[1,:,:].astype('uint8'), cmap='gray')
    plt.show()    